In [1]:
%pip install torch torchvision pandas scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.1 MB 1.1 MB/s eta 0:00:11
   ---------------------------------------- 0.1/11.1 MB 1.2 MB/s eta 0:00:10
   ---------------------------------------- 0.1/11.1 MB 1.2 MB/s eta 0:00:10
    --------------------------------------- 0.1/11.1 MB 711.9 kB/s eta 0:00:16
    --------------------------------------- 0.1/11.1 MB 711.9 kB/s eta 0:00:16
    --------------------------------------- 0.2/11.1 MB 655.9 kB/s eta 0:00:17
    --------------------------------------- 0.2/11.1 MB 655.9 kB/s eta 0:00:17
    --------------------------------------- 0.2/11.1 MB 655.9 kB/s eta 0:00:17
   - -------------------------------------- 0.3/11.1 MB 679.4 kB/s eta 0:00:16
   - -------------------------------------- 0.3/11.1 MB 726.4 kB/s eta 0:00:15
   - -------------------------------------- 0.4/11.1 MB 715.8 kB/s eta 0:00:


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import recall_score, average_precision_score
from PIL import Image

In [ ]:
# ======= 1. Load Dataset =======
df = pd.read_csv(r'D:\School\Semester 8\PENGANTAR DEEP LEARNING\UTS\dataset-G\multilabel_final.csv')
df['angle'] = df['filepath'].apply(lambda x: x.split('/')[1])  # e.g., 'angle-3'

# Encode angle label
le = LabelEncoder()
df['angle_encoded'] = le.fit_transform(df['angle'])

# Separate label sets
angle_labels = df['angle_encoded'].values
component_labels = df.iloc[:, 1:6].values  # front_left_door ... hood

# ======= 2. Image Dataset Class =======
class CarDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['filepath']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        angle = row['angle_encoded']
        components = torch.tensor(row.iloc[1:6].values.astype('float32'))
        return image, angle, components

# ======= 3. Image Transforms (Data Augmentation) =======
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),          # Randomly flip the image horizontally
    transforms.RandomRotation(15),               # Random rotation
    transforms.RandomVerticalFlip(),             # Random vertical flip
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

# ======= 4. Split Dataset =======
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = CarDataset(train_df, transform=transform)
val_dataset = CarDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Increased batch size
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# ======= 5. Model Definitions =======
# --- Model 1: Angle Classification (single label) ---
class AngleClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.resnet50(pretrained=True)  # Use ResNet50 for better performance
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# --- Model 2: Component Status Classification (multi-label) ---
class ComponentClassifier(nn.Module):
    def __init__(self, num_outputs):
        super().__init__()
        self.model = models.resnet50(pretrained=True)  # Use ResNet50 for better performance
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_outputs),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

angle_model = AngleClassifier(num_classes=len(le.classes_))
component_model = ComponentClassifier(num_outputs=5)

# ======= 6. Training Loop Function with Early Stopping and LR Scheduling =======
def train_one_epoch(model, dataloader, criterion, optimizer, scheduler=None, task='angle'):
    model.train()
    running_loss = 0.0
    for images, angles, components in dataloader:
        images = images.cuda()
        angles = angles.cuda()
        components = components.cuda()

        optimizer.zero_grad()
        outputs = model(images)
        if task == 'angle':
            loss = criterion(outputs, angles)
        else:
            loss = criterion(outputs, components)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    if scheduler:
        scheduler.step()  # Adjust learning rate based on scheduler

    return running_loss / len(dataloader)

# ======= 7. Evaluation Metrics Functions =======
def calculate_metrics(model, dataloader, task='angle'):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0.0

    with torch.no_grad():
        for images, angles, components in dataloader:
            images = images.cuda()
            angles = angles.cuda()
            components = components.cuda()

            outputs = model(images)
            if task == 'angle':
                loss = criterion_angle(outputs, angles)
                preds = torch.argmax(outputs, dim=1)  # Angle prediction (single label)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(angles.cpu().numpy())
            else:
                loss = criterion_components(outputs, components)
                preds = (outputs > 0.5).int()  # Multi-label component prediction (0 or 1)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(components.cpu().numpy())

            total_loss += loss.item()

    # Calculate metrics
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Accuracy for angle classification
    if task == 'angle':
        accuracy = np.mean(all_preds == all_labels)
        print(f"Accuracy: {accuracy:.4f}")
    else:
        accuracy = np.mean(np.all(all_preds == all_labels, axis=1))
        print(f"Accuracy (component): {accuracy:.4f}")

    # Recall for multi-label components
    if task == 'component':
        recall = recall_score(all_labels, all_preds, average='macro')
        print(f"Recall (component): {recall:.4f}")

        # Mean Average Precision (mAP)
        mAP = average_precision_score(all_labels, all_preds, average='macro')
        print(f"mAP (component): {mAP:.4f}")

    # Average loss
    avg_loss = total_loss / len(dataloader)
    print(f"Average Loss: {avg_loss:.4f}")
    
    return avg_loss

# ======= 8. Start Training =======
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
angle_model = angle_model.to(device)
component_model = component_model.to(device)

# Angle: CrossEntropyLoss
criterion_angle = nn.CrossEntropyLoss()
optimizer_angle = optim.Adam(angle_model.parameters(), lr=1e-4)

# Components: BCEWithLogitsLoss (multi-label)
criterion_components = nn.BCELoss()
optimizer_components = optim.Adam(component_model.parameters(), lr=1e-4)

# Learning rate scheduler
scheduler_angle = optim.lr_scheduler.StepLR(optimizer_angle, step_size=5, gamma=0.5)  # Reduce LR after every 5 epochs
scheduler_components = optim.lr_scheduler.StepLR(optimizer_components, step_size=5, gamma=0.5)

# Early stopping setup
best_val_loss = float('inf')
patience = 3
epochs_without_improvement = 0

for epoch in range(20):  # Increased number of epochs
    print(f"Epoch {epoch+1}/20")
    
    # Training phase
    train_loss_angle = train_one_epoch(angle_model, train_loader, criterion_angle, optimizer_angle, scheduler_angle, task='angle')
    train_loss_component = train_one_epoch(component_model, train_loader, criterion_components, optimizer_components, scheduler_components, task='component')
    
    print(f"Train Loss Angle: {train_loss_angle:.4f}, Train Loss Component: {train_loss_component:.4f}")

    # Validation phase
    val_loss_angle = calculate_metrics(angle_model, val_loader, task='angle')
    val_loss_component = calculate_metrics(component_model, val_loader, task='component')

    print(f"Validation Loss Angle: {val_loss_angle:.4f}, Validation Loss Component: {val_loss_component:.4f}")

    # Early stopping check
    if val_loss_angle + val_loss_component < best_val_loss:
        best_val_loss = val_loss_angle + val_loss_component
        epochs_without_improvement = 0
        # Save the best model
        torch.save({
            'angle_model_state_dict': angle_model.state_dict(),
            'component_model_state_dict': component_model.state_dict(),
            'angle_label_encoder': le.classes_.tolist()
        }, 'best_car_multi_model.pt')
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print("Early stopping triggered. No improvement in validation loss.")
        break

# ======= Save Final Models =======
torch.save({
    'angle_model_state_dict': angle_model.state_dict(),
    'component_model_state_dict': component_model.state_dict(),
    'angle_label_encoder': le.classes_.tolist()  # optional: saving the label classes
}, 'car_multi_model_2.pt')

d:\School\Semester 8\PENGANTAR DEEP LEARNING\UTS\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\School\Semester 8\PENGANTAR DEEP LEARNING\UTS\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
